<a href="https://colab.research.google.com/github/coldsober-irene/AIF-machine-learning/blob/main/imageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

In [ ]:
drive.mount("/content/drive")

In [15]:
# LIBRARIES NEEDED
import os 
import imghdr
import cv2
from matplotlib import pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
import numpy as np

In [16]:
# GATHER IMAGES
image_path = "/content/drive/MyDrive/data_for_study/images/class2"
classes = ["irene", "messi", "mbappe", "neymar"]
extensions = ['jpeg', 'jpg', 'png', 'bmp']

In [17]:
# REMOVE DOGGY IMAGES
[os.remove(os.path.join(image_path, class_, im)) for class_ in classes for im in os.listdir(os.path.join(image_path, class_)) if imghdr.what(os.path.join(image_path, class_, im)) not in extensions]
all_images = {}
for class_ in classes:
    all_images[class_] = [os.path.join(os.path.join(image_path, class_, im)) for im in os.listdir(os.path.join(image_path, class_))]

In [6]:
# CREATE DATASETS
datasets = tf.keras.utils.image_dataset_from_directory(image_path) # this resize image to 256x256 and the batch size of 32
# print("available batches in whole datasets : ", len(datasets)) # this helps when splitting the datasets (train, test & validate)
# resize the pixel and move from being 0 to 255 and become 0 to 1: this minimize training data & computational cost
datasets = datasets.map(lambda x, y: (x / 255, y))
# to be able to access theses datasets
# datasets =  datasets.as_numpy_iterator()

Found 247 files belonging to 2 classes.


In [7]:
# SPLIT DATA
train_size = int(len(datasets) * .7) # these are batches
test_size = int(len(datasets) * .1) + 1 # these are batches
valid_size = int(len(datasets) * .2) # these are batches

# Now take the exact batches in the datasets based on the number of batches for each category 
train = datasets.take(train_size) 
valid = datasets.skip(train_size).take(valid_size)
test = datasets.skip(train_size + valid_size).take(test_size)

In [ ]:
# MODEL CREATION
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation = 'relu', input_shape = (256,256,3)))
model.add(MaxPooling2D())

model.add(Conv2D(32, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())

model.add(Conv2D(16, (3,3), 1, activation = 'relu'))
model.add(MaxPooling2D())

# convert 2D into 1D
model.add(Flatten())

# HIDDEN LAYERS (FULL CONNECTED LAYERS)
model.add(Dense(256, activation = 'relu'))
model.add(Dense(1, activation = "softmax"))

# COMPILE THE MODEL
model.compile('adam', loss = tf.losses.CategoricalCrossentropy(), metrics = ['accuracy'])

# see the summary of the model
print(model.summary())

In [14]:
# TRAIN THE MODEL
log_dir = "/model_logs"
try:
  os.mkdir("/model_logs")
except Exception:
  pass 
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir) # this is for tracking the performance of the model during training
hist = model.fit(train, epochs = 11, validation_data = valid, callbacks = [tensorboard_callback])

Epoch 1/11
5/5 [==============================] - 18s 716ms/step - loss: 0.0000e+00 - accuracy: 0.2875 - val_loss: 0.0000e+00 - val_accuracy: 0.2812
Epoch 2/11
5/5 [==============================] - 4s 537ms/step - loss: 0.0000e+00 - accuracy: 0.2812 - val_loss: 0.0000e+00 - val_accuracy: 0.4688
Epoch 3/11
5/5 [==============================] - 5s 692ms/step - loss: 0.0000e+00 - accuracy: 0.3125 - val_loss: 0.0000e+00 - val_accuracy: 0.1875
Epoch 4/11
5/5 [==============================] - 5s 693ms/step - loss: 0.0000e+00 - accuracy: 0.2812 - val_loss: 0.0000e+00 - val_accuracy: 0.3750
Epoch 5/11
5/5 [==============================] - 5s 690ms/step - loss: 0.0000e+00 - accuracy: 0.2688 - val_loss: 0.0000e+00 - val_accuracy: 0.2500
Epoch 6/11
5/5 [==============================] - 4s 540ms/step - loss: 0.0000e+00 - accuracy: 0.2812 - val_loss: 0.0000e+00 - val_accuracy: 0.1250
Epoch 7/11
5/5 [==============================] - 5s 695ms/step - loss: 0.0000e+00 - accuracy: 0.2625 - val_los

In [ ]:
# MAKE PREDICTION
im1 = cv2.imread("/content/drive/MyDrive/data_for_study/file_from_db.png")
im2 = cv2.imread("/content/drive/MyDrive/data_for_study/mesi.jpg")
im11 = cv2.imread("/content/drive/MyDrive/others/WIN_20210709_115246.JPG")
im22 = cv2.imread("/content/drive/MyDrive/data_for_study/messii.jpg")
chris = cv2.imread("/content/drive/MyDrive/data_for_study/Cristiano_Ronaldo_2018.jpg")
other = cv2.imread("/content/drive/MyDrive/data_for_study/photo-1519238263530-99bdd11df2ea.jpg")

def process_img(imgs = []):
  processed = []
  for im in imgs:
    im = tf.image.resize(im, (256, 256))
    processed.append(np.expand_dims(im/255, 0))
  return processed
# CALL
images = process_img(imgs = [im1, im2, im11, im22, chris, other])

im1_pred = model.predict(images[0])
im11_pred = model.predict(images[2])
im2_pred = model.predict(images[1])
im22_pred = model.predict(images[3])
chris_pred = model.predict(images[-2])
other_pred = model.predict(images[-1])

print("irene prediction: ", im1_pred, im11_pred)
print("messi prediction : ", im2_pred, im22_pred)
print("christiano prediction: ", chris_pred)
print("other prediction: ", other_pred)

In [10]:
tf.keras.layers.MaxPooling2D??